In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
try:
    get_ipython
    from tqdm import tqdm_notebook as tqdm
except NameError:
    from tqdm import tqdm

def get_url(url):
    response = get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    return soup

def get_elements():
    return((driver.find_elements_by_xpath("//p[@class='searchtbp1']//a")))

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import re
chrome_driver = 'C:/Users/ndevito/Dropbox/Python projects/Drivers/chromedriver.exe'

import os
from pathlib import Path
import platform

platform = platform.platform()
cwd = os.getcwd()
download_path = os.path.join(cwd,'Rebec Downloads')
#adjust this to fit your specific file structure 
parent = str(Path(cwd).parents[0]) 

In [2]:
if "Darwin" in platform:
    chrome_driver = os.path.join(parent, 'Drivers', 'chromedriver')
elif "Windows" in platform:
    chrome_driver = os.path.join(parent, 'Drivers', 'chromedriver.exe')
else:
    print("No OS/Chromedriver match. OS: {}".format(platform))

In [3]:
try:
    chromeOptions = webdriver.ChromeOptions()
    prefs = {'disk-cache-size': 4096,
            'safebrowsing.enabled': 'false'}
    chromeOptions.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(executable_path=chrome_driver, options=chromeOptions)
    driver.get('http://cris.nih.go.kr/cris/en/search/basic_search.jsp')
    select = Select(driver.find_element_by_id('rowcount'))
    select.select_by_value('50')
    driver.find_element_by_xpath("//input[@alt='Search']").click()
    total_trials = (driver.find_element_by_xpath("//strong[@class='searchtbtnb']")).text
    onclick_el = (driver.find_elements_by_xpath("//a[@href='#paging']"))[-1].get_attribute('onclick')
    max_page = onclick_el[onclick_el.find("(")+1:onclick_el.find(")")]

    url_suff_list = []
    pattern = re.compile(r'\d+$')
    for a in get_elements():
        url_suff_list.append(re.search(pattern, a.get_attribute('href')).group())
    next_page = 2
    while next_page < int(max_page)+1:
        (driver.find_element_by_xpath("//a[@onclick ='return jf_page({});']".format(next_page))).click()
        for a in get_elements():
            url_suff_list.append(re.search(pattern, a.get_attribute('href')).group())
        next_page += 1
    driver.quit()
except Exception as e:
    driver.quit()
    print(e)

In [4]:
int(total_trials) == len(url_suff_list)

True

In [5]:
default_cris_url = 'https://cris.nih.go.kr/cris/en/search/search_result_st01.jsp?seq='

In [22]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from datetime import date

def str_to_date(datestr):  
    is_defaulted_date = False
    #this fixes a specific issie with one trial with an impossible date for a primart completion date (31 June 2017)
    #https://cris.nih.go.kr/cris/en/search/search_result_st01.jsp?seq=7625
    #based on the provided full completion date in that study, we have defaulted to 30 June 2017
    if datestr == '2017-06-31':
        parsed_date = date(2017,6,30)
        is_defaulted_date = True
    #this fixes an issue with a trial with a bad date format (no hyphens) in the start date field
    #https://cris.nih.go.kr/cris/en/search/search_result_st01.jsp?seq=3990
    elif datestr == '20140508':
        parsed_date = date(2014,5,8)
        is_defaulted_date = True
    elif datestr not in ['-', 'Actual', 'Anticipated', '']:
        try:
            parsed_date = datetime.strptime(datestr, '%Y-%m-%d').date()
        except ValueError:
            parsed_date = datetime.strptime(datestr, '%Y-%m').date() + relativedelta(months=+1) - timedelta(days=1)
            is_defaulted_date = True
    else:
        parsed_date = None
    return (parsed_date, is_defaulted_date)

In [23]:
all_cris_trials = []
for suff in tqdm(url_suff_list):
    t_d = {}
    soup = get_url(default_cris_url + suff)
    t_d['trial_id'] = soup.find(text=re.compile(r'CRIS Registration Number')).findNext('td').text.strip()
    t_d['recruitment_status'] = soup.find(text=re.compile(r'Overall Recruitment Status')).findNext('td').text.strip()
    for x in ['Terminated', 'Suspended', 'Withdrawn']:
        if x in t_d['recruitment_status']:
            t_d['recruitment_status'] = x
            break 
    t_d['start_date'] = str_to_date(soup.find(text=re.compile(r'Date of First Enrollment')).findNext('td').text.strip().split(',')[0].strip())[0]
    t_d['primary_completion_date'] = str_to_date(soup.find(text=re.compile(r'Primary Completion Date')).findNext('td').text.strip().split(',')[0].strip())[0]
    t_d['study_completion_date'] = str_to_date(soup.find(text=re.compile(r'Study Completion Date')).findNext('td').text.strip().split(',')[0].strip())[0]
    results_info = {}
    results_registered = {}
    if soup.find(text=re.compile(r'Result Registerd')).findNext('td').text.strip() == 'No' or not soup.find(text=re.compile(r'Result Registerd')).findNext('td').text.strip():
        results_registered['has_results'] = 'No'
        results_info['results_registered'] = results_registered
    else:
        for t in soup.find(text=re.compile(r'Result Registerd')).findNext('td').text.strip().split('\r\n\t'):
            if t.strip() == '':
                continue
            elif t.strip() == 'Yes':
                results_registered['has_results'] = 'Yes'
            elif t.strip() == 'Published':
                results_registered['published'] = 'Yes'
            elif t.strip() == 'Results Upload':
                results_registered['uploaded'] = 'Yes'
        results_info['results_registered'] = results_registered

        publications = {}
        if soup.find(text=re.compile(r'Number of Publication')).findNext('td').text.strip() == '0':
            pass
        else:
            publications['number_of_pubs'] = soup.find(text=re.compile(r'Number of Publication')).findNext('td').text.strip()
            for pub in list(range(1,int(soup.find(text=re.compile(r'Number of Publication')).findNext('td').text.strip())+1)):
                publications['publication_{}'.format(pub)] = soup.find(text=re.compile(r'Publications {}'.format(pub))).findNext('td').text.strip().replace('\t','').replace('\xa0','').replace('\r','').replace('\n','')
            results_info['publications'] = publications

        if soup.find(text=re.compile(r'Results Upload')).findNext('a').get('href'):
            results_info['uploaded_document'] = 'https://cris.nih.go.kr/cris' + soup.find(text=re.compile(r'Results Upload')).findNext('a').get('href').replace('../..','')

        if soup.find(text=re.compile(r'Date of Posting Results')).findNext('td').text.strip():
            results_info['results_posted_date'] = str_to_date(soup.find(text=re.compile(r'Date of Posting Results')).findNext('td').text.strip())[0]

        if soup.find(text=re.compile(r'Protocol URL or File Upload')).findNext('td').text.strip():
            if '../..' in soup.find(text=re.compile(r'Protocol URL or File Upload')).findNext('a').get('href'):
                results_info['protocol'] = 'https://cris.nih.go.kr/cris' + soup.find(text=re.compile(r'Protocol URL or File Upload')).findNext('a').get('href').replace('../..','')
            else:
                results_info['protocol'] = soup.find(text=re.compile(r'Protocol URL or File Upload')).findNext('a').get('href')

        if soup.find(text=re.compile(r'Brief Summary')).findNext('td').text.strip():
            results_info['brief_summary'] = soup.find(text=re.compile(r'Brief Summary')).findNext('td').text.strip()
    t_d['results_info'] = results_info
    all_cris_trials.append(t_d)

In [24]:
len(all_cris_trials)

4430

In [27]:
import csv
from datetime import date
import json

def cris_csv():
    with open('cris_scrape - ' + str(date.today()) + '.csv','w', newline = '')as cris_csv:
        writer=csv.writer(cris_csv)
        for val in all_cris_trials:
            writer.writerow([json.dumps(val, indent=4, sort_keys=True, default=str)])

In [28]:
cris_csv()